# kaggle上的一个表情识别
（具体资料链接，见文末扩展资料）



数据形式：CSV存储，第一个是label，第二个是数据，第三个是阶段

网络：
用了两种网络，lenet-5改和CliqueNet，后者特殊一些。

数据预处理是有用的尝试，将脸居中等.


# 其他各种ipynb
基本都是对各个文件和过程的分解和调试




# 把每一部分都单独开一个笔记，下边的不继续了。

## fer_forward     lenet5变体网络结构定义

dropout只有0.95，不是很追求泛化的一个数值

lenet-5是7层网络，两个Conv+pooling循环，两个FC，一个OUTPUT。
改成9层网络，加了一个conv+pooling循环。卷积核还是5.
都是5*5的核

In [1]:
import tensorflow as tf

In [2]:
#需要预处理数据，生成tfrecord
import fer_generator
import fer_forward
import fer_backward
import fer_config as config

In [ ]:
fer_generator.devide_train_data()

In [2]:


#待会计算一下，看看他输入是什么样，对应输入的每层的情况算一下，因为他的第三个conv提升了通道数。


In [ ]:

#原图需要处理成单通道，lenet的input是单通道的。

fer_backward.backward()

In [8]:
ls '/home/qw/Documents/datasets/'

fer2013/  FERPlus/


# 读取tfrecord生成信息队列的测试。
## 没有对应文件的情况
只是想测一下地址为空的时候的反馈
看来这么一点小操作都听繁琐的，照猫画虎全程不犯错很难把工程弄的很透。
这里主要问题就是，如果你没有tfrecord，地址对应的文件没有，直接训练，不会有任何报错。
尝试方法1:直接把tensor拿到，sess.run，结果不行，因为是要用一个producer的，现在等于阻塞了，producer没跑起来。必须启动线程协调器才能得到NotFoundError错误提示，那个producer自己动不起来，启动线程协调器就行了，不用显式的调用producer本身，也类似扔到一个集合，自动给启动了吧，更细的说明，官方没给出，只能看实现。

In [3]:

#随便拿几个数据试一下没tfrecord文件时接口的反馈
#tf.train.shuffle_batch阻塞的，没什么反馈,
# img_batch, label_batch = fer_generator.get_tfrecord(10,config.tfRecord_train)
#只拿tensor，不过shuffle，都是阻塞，太不友好。。。。。。。
#img, label = fer_generator.get_tfrecord0(10,config.tfRecord_train)
# print(img)
# with tf.Session() as sess:
#     print(sess.run(img))
#     print(sess.run(label))


#因为read_tfRecord是要走string_input_producer的
filename_queue = tf.train.string_input_producer([config.tfRecord_train])
print(filename_queue)

reader = tf.TFRecordReader()
print('reader:',reader)
_,serialized_example = reader.read(filename_queue)
print(serialized_example)
features = tf.parse_single_example(serialized_example,
                                   features={
                                       'label':tf.FixedLenFeature([7],tf.int64),
                                       'img_raw':tf.FixedLenFeature([],tf.string)
                                             })
print(features)
img = tf.decode_raw(features['img_raw'],tf.uint8)
img.set_shape([config.img_height * config.img_height])

img = tf.cast(img,tf.float32)*(1./255)
label = tf.cast(features['label'],tf.float32)
print(img)
with tf.Session() as sess:
    #直接运行阻塞
#     print(sess.run(img))
#     print(sess.run(label))

    # 创建一个线程协调器
    coord=tf.train.Coordinator()
    # 启动入队线程
    threads=tf.train.start_queue_runners(sess=sess,coord=coord)
    print(sess.run(img))
    print(sess.run(label))  
    
    coord.request_stop()
    coord.join(threads)
    
#这样就可以得到错误提示了
# NotFoundError: /home/qw/Documents/datasets/fer2013/fer2013_train.tfrecords

reader: <tensorflow.python.ops.io_ops.TFRecordReader object at 0x7f9043044a58>
Tensor("ReaderReadV2:1", shape=(), dtype=string)
{'img_raw': <tf.Tensor 'ParseSingleExample/Squeeze_img_raw:0' shape=() dtype=string>, 'label': <tf.Tensor 'ParseSingleExample/Squeeze_label:0' shape=(7,) dtype=int64>}
Tensor("mul:0", shape=(2304,), dtype=float32)
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: input_producer/input_producer_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](input_producer, input_producer/RandomShuffle)]]


NotFoundError: /home/qw/Documents/datasets/fer2013/fer2013_train.tfrecords
	 [[Node: ReaderReadV2 = ReaderReadV2[_device="/job:localhost/replica:0/task:0/cpu:0"](TFRecordReaderV2, input_producer)]]

Caused by op 'ReaderReadV2', defined at:
  File "/home/qw/anaconda3/envs/py36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/qw/anaconda3/envs/py36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/qw/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/qw/anaconda3/envs/py36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/qw/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/qw/anaconda3/envs/py36/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 1

Help on function string_input_producer in module tensorflow.python.training.input:

string_input_producer(string_tensor, num_epochs=None, shuffle=True, seed=None, capacity=32, shared_name=None, name=None, cancel_op=None)
    Output strings (e.g. filenames) to a queue for an input pipeline.
    
    Note: if `num_epochs` is not `None`, this function creates local counter
    `epochs`. Use `local_variables_initializer()` to initialize local variables.
    
    Args:
      string_tensor: A 1-D string tensor with the strings to produce.
      num_epochs: An integer (optional). If specified, `string_input_producer`
        produces each string from `string_tensor` `num_epochs` times before
        generating an `OutOfRange` error. If not specified,
        `string_input_producer` can cycle through the strings in `string_tensor`
        an unlimited number of times.
      shuffle: Boolean. If true, the strings are randomly shuffled within each
        epoch.
      seed: An integer (optional)

# 扩展资料

原问题和数据集来源
https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data
kaggle提供了data，不过好像数据不算很好，有一个微软标注的FERPlus数据集可以做代替。

这个代码来源
https://github.com/elf-G/fer2013
    
扩展数据来源
https://github.com/Microsoft/FERPlus


原问题的讨论：
很多铜牌发言，拿到奖金的也是华人
https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/discussion